In [1]:
! pip install transformers
!pip install tqdm
import numpy as np 
import pandas as pd 



from torch.optim import AdamW

from transformers import get_linear_schedule_with_warmup 

from sklearn import preprocessing 

import os



import transformers 

import torch.nn as nn

from tqdm.notebook import tqdm

import torch 

from sklearn.model_selection import train_test_split 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 13.9 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
     |████████████████████████████████| 596 kB 64.1 MB/s 
     |████████████████████████████████| 6.6 MB 54.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Dataset/ner_dataset.csv",encoding="latin-1")

In [ ]:
class Config:
    MAX_LENGTH=128
    bert_model='bert-base-uncased'
    TOKENIZER=transformers.BertTokenizer.from_pretrained(bert_model, do_lower_case=True)
    TRAIN_BATCH_SIZE=32
    VAL_BATCH_SIZE=18   
    EPOCHS=3
    

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
Config.TOKENIZER.encode_plus("hI HELLO")

{'input_ids': [101, 7632, 7592, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [ ]:
data.loc[:,"Sentence #"]=data["Sentence #"].fillna(method="ffill")

In [ ]:
enc_pos = preprocessing.LabelEncoder()
enc_tag = preprocessing.LabelEncoder()

data.loc[:, "POS"] = enc_pos.fit_transform(data["POS"])
data.loc[:, "Tag"] = enc_tag.fit_transform(data["Tag"])

sentences = data.groupby("Sentence #")["Word"].apply(list).values
pos = data.groupby("Sentence #")["POS"].apply(list).values
tag = data.groupby("Sentence #")["Tag"].apply(list).values


In [ ]:
len(list(enc_pos.classes_))

42

In [ ]:
len(list(enc_tag.classes_))

17

In [ ]:
pos

array([list([19, 10, 19, 35, 34, 10, 17, 29, 31, 7, 16, 10, 17, 5, 31, 7, 16, 10, 11, 19, 10, 7, 16, 2]),
       list([11, 19, 35, 22, 35, 29, 31, 16, 29, 11, 11, 19, 10, 7, 16, 17, 1, 10, 7, 17, 16, 16, 36, 33, 2]),
       list([16, 19, 17, 32, 11, 19, 10, 7, 17, 11, 16, 1, 40, 11, 17, 19, 35, 34, 29, 31, 34, 29, 31, 7, 12, 11, 16, 10, 11, 17, 17, 2]),
       ...,
       list([33, 17, 21, 11, 17, 6, 19, 1, 19, 19, 32, 19, 10, 19, 32, 34, 10, 19, 10, 16, 19, 10, 16, 1, 11, 19, 2]),
       list([10, 24, 1, 19, 35, 34, 11, 19, 10, 7, 34, 5, 34, 29, 31, 11, 19, 10, 7, 16, 2]),
       list([7, 17, 17, 36, 33, 7, 16, 10, 11, 19, 29, 31, 16, 5, 16, 19, 10, 11, 17, 1, 33, 7, 16, 35, 41, 10, 23, 16, 10, 16, 2, 41])],
      dtype=object)

In [ ]:
class NERDataset:
    def __init__(self,sentence,pos,tag):
        self.sentence=sentence
        self.pos=pos
        self.tag=tag
    def __len__(self):
        return len(self.sentence)
         
    def __getitem__(self,item):
        sentence=self.sentence[item]
        pos=self.pos[item]
        tag=self.tag[item]     
        ids=[]
        target_pos=[]
        target_tag=[]



        
        for i , data in enumerate(sentence):
            id=Config.TOKENIZER.encode(data,add_special_tokens=False)
            
            len_ids=len(id)
           
            ids.extend(id)
            target_pos.extend(len_ids*[pos[i]])
            target_tag.extend(len_ids*[tag[i]])



   
        if len(ids) > Config.MAX_LENGTH-2:
            ids=[101]+ids[:Config.MAX_LENGTH-2]+[102]
            target_pos=[0]+target_pos[:Config.MAX_LENGTH-2]+[0]
            target_tag=[0]+target_tag[:Config.MAX_LENGTH-2]+[0]
            mask=[1]*len(ids)
          
        else:
            ids=[101]+ids+[102]
            target_pos=[0]+target_pos+[0]
            target_tag=[0]+target_tag+[0]
            mask=[1]*len(ids)
        # Since there is no different segments so making it as zero complete length for token type ids 
        token_type_ids=[0]*Config.MAX_LENGTH  
        padding_len=Config.MAX_LENGTH-len(ids)
        
        ids=ids+([0]*padding_len)
        mask=mask+([0]*padding_len)
        target_pos=target_pos+([0]*padding_len)
        target_tag=target_tag+([0]*padding_len)
        

       
        
        
        return {
            
            'ids':torch.tensor(ids,dtype=torch.long),
            'mask':torch.tensor(mask,dtype=torch.long),
            'target_pos':torch.tensor(target_pos,dtype=torch.long),
            'target_tag':torch.tensor(target_tag,dtype=torch.long),
            'token_type_ids':torch.tensor(token_type_ids,dtype=torch.long)
            
            
        }   

In [ ]:
class CrossEntropyLossManual:
    """
    y0 is the vector with shape (batch_size,C)
    x shape is the same (batch_size), whose entries are integers from 0 to C-1
    """

    def __init__(self, ignore_index=-100) -> None:
        self.ignore_index = ignore_index

    def __call__(self, y0, x):
        loss = 0.
        n_batch, n_class = y0.shape
        for y1, x1 in zip(y0, x):
            class_index = int(x1.item())

            print(class_index)

            if class_index == self.ignore_index:
                n_batch -= 1
                continue
            loss = loss + torch.log(torch.exp(y1[class_index]) / (torch.exp(y1).sum()))
        loss = - loss / n_batch
        return loss



In [ ]:
def loss_fn(predicited, actual):
    # ignore index helps us not to include those values ot gradient 
    loss_cal=CrossEntropyLossManual(ignore_index=0)


    predicited_values=predicited.view(-1,predicited.size(-1))



    actual_values=actual.view(-1)

    loss=loss_cal(predicited_values,actual_values)
    return loss
    

In [ ]:
from transformers import AutoModel , AutoConfig
config=AutoConfig.from_pretrained(Config.bert_model)

config.attention_probs_dropout_prob=0.3
config.hidden_dropout_prob=0.3


In [ ]:
class NERModel(nn.Module):
    def __init__(self,num_tag,num_pos,config):
        super(NERModel,self).__init__()
        self.bert_model=AutoModel.from_pretrained(Config.bert_model,config=config)
        self.num_tag=num_tag
        self.num_pos=num_pos
        self.pos_layer=nn.Linear(768,self.num_pos,bias=False)
        self.tag_layer=nn.Linear(768,self.num_tag,bias=False)
        self.dropout_1=nn.Dropout(0.3)
        self.dropout_2=nn.Dropout(0.3)
     
    def forward(self,ids,mask,target_pos,target_tag,token_type_ids):
        # Because it return (batch_lenght , sequence , dimension of the each sequnce) 


      
        
        output_bert=self.bert_model(ids,attention_mask=mask,token_type_ids=token_type_ids)['last_hidden_state']

        tags=self.dropout_1(output_bert)

        pos=self.dropout_2(output_bert)
        
        out_pos=self.pos_layer(pos)

        out_tags=self.tag_layer(tags)
        
        loss_tag = loss_fn(out_tags, target_tag)

      

        loss_pos = loss_fn(out_pos, target_pos)


        loss = (loss_tag + loss_pos) / 2

        return loss ,out_tags, out_pos
        

In [ ]:
DEVICE=torch.device('cuda' if torch.cuda.is_available() else'cpu')

In [ ]:
def train_fn(data_loader,model,optimizer,scheduler=None):
    train_loss=0
    
    model.train()
    
    train_data=tqdm(data_loader,total=len(data_loader))
    
    for batch_size, data in enumerate(train_data):
        
        ids=data['ids']
        mask=data['mask']
        token_type_ids=data['token_type_ids']
        target_pos=data['target_pos']
        target_tag=data['target_tag']
      
    
        
        # loading to device    
        
        ids=ids.to(DEVICE,dtype=torch.long)
        mask=mask.to(DEVICE,dtype=torch.long)
        token_type_ids=token_type_ids.to(DEVICE,dtype=torch.long)
        target_pos=target_pos.to(DEVICE,dtype=torch.long)
        target_tag=target_tag.to(DEVICE,dtype=torch.long)
        
        optimizer.zero_grad()
        
        loss,_,_=model(ids,mask,target_pos,target_tag,token_type_ids)
        
        # to calculate weight w.r.t loss
        loss.backward()
        
        # to update the weight 
        
        optimizer.step()
        
        # to increase/decerease the learning 
        
        scheduler.step()
        
        train_loss=+loss.item()
        
    return train_loss/len(data_loader)    
        
        
def eval_fn(eval_loader,model):
    model.eval()
    final_eval_loss=0
    tdqm=tqdm(eval_loader,total=len(eval_loader))
    
    for batch_size, data in enumerate(tdqm):
        ids=data['ids']
        mask=data['mask']
        token_type_ids=data['token_type_ids']
        target_pos=data['target_pos']
        target_tag=data['target_tag']
        
        # loading to device    
        
        ids=ids.to(DEVICE,dtype=torch.long)
        mask=mask.to(DEVICE,dtype=torch.long)
        token_type_ids=token_type_ids.to(DEVICE,dtype=torch.long)
        target_pos=target_pos.to(DEVICE,dtype=torch.long)
        target_tag=target_tag.to(DEVICE,dtype=torch.long)
        
        with torch.no_grad():
            loss,_,_=model(ids,mask,target_pos,target_tag,token_type_ids)
            
            final_eval_loss=+loss.item()
            
    return final_eval_loss/len(eval_loader)        

In [ ]:
# Intializing the model 

num_pos=len(list(enc_pos.classes_))

num_tags=len(list(enc_tag.classes_))



In [ ]:
model=NERModel(num_tags,num_pos,config)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model=model.to(DEVICE)

In [ ]:
df_train_sentence,df_valid_sentence,df_train_tag,df_valid_tag,df_train_pos,df_valid_pos= train_test_split(sentences,tag,pos,test_size=0.3, random_state=42)
    
train_data=NERDataset(df_train_sentence,df_train_pos,df_train_tag)

In [ ]:
def run():
    df_train_sentence,df_valid_sentence,df_train_tag,df_valid_tag,df_train_pos,df_valid_pos= train_test_split(sentences,tag,pos,test_size=0.3, random_state=42)
    
    train_data=NERDataset(df_train_sentence,df_train_pos,df_train_tag)
    
    val_data=NERDataset(df_valid_sentence,df_valid_pos,df_valid_tag)
    
    train_data_loader=torch.utils.data.DataLoader(
        train_data,
        batch_size=32
    )
    
    valid_data_loader=torch.utils.data.DataLoader(
        val_data,
        batch_size=16
    )
    
    # create of optimizer 
    num_train_steps=len(df_train_sentence)/Config.TRAIN_BATCH_SIZE*Config.EPOCHS

    
    
    parameters=list(model.named_parameters())
    
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    
    
    # we dont want to apply weight deccay to above mentioned weights loss+ weight deccay * l2/l2 norma 
    
    optimized_paramater=[
        
            {
        "params": [
            p for n, p in parameters if not any(
                nd in n for nd in no_decay
            )
        ],
        "weight_decay": 0.001,
    },    {
        "params": [
            p for n, p in parameters if any(
                nd in n for nd in no_decay
            )
        ],
        "weight_decay": 0.001,
    },
        
        
        
    ]
    
    optimizer=AdamW(optimized_paramater,lr=1e-5)
    
    scheduler=get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    
    )
    
    path='/content/drive/MyDrive/NERModel'


 
    
    for x in range(Config.EPOCHS):
        print(f' Current epoch value was {x+1} out of {Config.EPOCHS}')

        
        loss_train = train_fn(train_data_loader,model,optimizer,scheduler)
        
        loss_val = eval_fn( valid_data_loader,model)
        
        print(f' The epoch {x+1} encountered a  train loss of : {loss_train} and validation loss of {loss_val}')  

        torch.save(model,path)

In [ ]:
run()

 Current epoch value was 1 out of 3


  0%|          | 0/1050 [00:00<?, ?it/s]

  0%|          | 0/900 [00:00<?, ?it/s]

 The epoch 1 encountered a  train loss of : 0.00019253644205275035 and validation loss of 0.00022983991437488133
 Current epoch value was 2 out of 3


  0%|          | 0/1050 [00:00<?, ?it/s]

  0%|          | 0/900 [00:00<?, ?it/s]

 The epoch 2 encountered a  train loss of : 0.00011900317101251512 and validation loss of 0.00020744419760174222
 Current epoch value was 3 out of 3


  0%|          | 0/1050 [00:00<?, ?it/s]

  0%|          | 0/900 [00:00<?, ?it/s]

 The epoch 3 encountered a  train loss of : 0.00016823158377692813 and validation loss of 0.00021035331818792556
